<h2>Import Libraries


<h2> Loading data

<h2> Train Test split

<h2> Creating datasets and dataloaders

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import numpy as np

import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizer

from dataset import *
from model import MyClassificator, BertForClassification, TransformerBasedModel
from trainer import Trainer, test_model

In [5]:
PATH = "datasets/"
MAX_LEN = 128
BATCH_SIZE = 128
EMBEDDING_SIZE = 128

In [29]:
train_data = pd.read_csv(os.path.join(PATH, "train.csv"))
test_data = pd.read_csv(os.path.join(PATH, "test.csv"))

In [30]:
train_data.head()


,movie_name,target,movie_description,id
0,Furies,0,Three furious vigilantes unite to take down a ...,133529636342330622371894152500993949030
1,RRR,0,The story of freedom fighters Komaram Bheem an...,133529660110779376651195430564179049830
2,John Wick,0,Legendary assassin John Wick (Keanu Reeves) re...,133529680710101630359923204885606137190
3,John Wick: Chapter 3 -- Parabellum,0,After gunning down a member of the High Table ...,133529687048354631501070212369122164070
4,Top Gun: Maverick,0,After more than thirty years of service as one...,133529699724860633783364227336154217830


In [36]:
train_data['id'] = train_data['id'].astype(np.float128)
print(train_data['id'])
m = train_data['id'].min()
train_data['id'] = (train_data['id'] - m)/(10.0**30)
train_data['id']
train_data["target"]

0       0.000000e+00
1       2.376845e-59
2       4.436777e-59
3       5.070602e-59
4       6.338253e-59
            ...     
8045    6.524915e-56
8046    6.526103e-56
8047    6.526658e-56
8048    6.527291e-56
8049    6.527846e-56
Name: id, Length: 8050, dtype: float128


0       0
1       0
2       0
3       0
4       0
       ..
8045    5
8046    5
8047    5
8048    5
8049    5
Name: target, Length: 8050, dtype: int64

In [8]:
#import matplotlib.pyplot as plt
#plt.hist([t for t in train_data.target], bins=20);
#plt.hist([len(s) for s in train_data.movie_description], bins=20);

In [9]:
train_split, val_split = train_test_split(train_data, train_frac=0.9)

In [10]:
tokenizer = DistilBertTokenizer.from_pretrained(
    "distilbert-base-uncased", truncation=True, do_lower_case=True)

In [11]:
len(tokenizer.vocab)

30522

In [12]:
train_dataset = RottenTomatoesDataset(train_split, tokenizer, MAX_LEN)
val_dataset = RottenTomatoesDataset(val_split, tokenizer, MAX_LEN)
test_dataset = RottenTomatoesDataset(test_data, tokenizer, MAX_LEN)

In [13]:
train_params = {"batch_size": BATCH_SIZE,
                "shuffle": True,
                "num_workers": 10
                }

test_params = {"batch_size": BATCH_SIZE,
               "shuffle": False,
               "num_workers": 10
               }

train_dataloader = DataLoader(train_dataset, **train_params)
val_dataloader = DataLoader(val_dataset, **test_params)
test_dataloader = DataLoader(test_dataset, **test_params)

<h2> Loading pretrained model from Huggingface

In [14]:
config = {
    "num_classes": 6,
    "dropout_rate": 0.0,
    "embedding_size": EMBEDDING_SIZE
}
model = BertForClassification(
    "distilbert-base-uncased",
    config=config
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
n_unfreezed = 0
for name, p in model.named_parameters():
    #print(name)
    if (
        name.startswith("classifier") or
        name.startswith("bert.transformer.layer.5") 
        ):
        
        p.requires_grad = True
        n_unfreezed += 1
    else:
        p.requires_grad = False

print(n_unfreezed) 

36


In [16]:
#test_model(model, train_dataloader)

<h2> Creating Trainer object and fitting the model

In [17]:
trainer_config = {
    "lr": 3e-4,
    "n_epochs": 2,
    "batch_size": BATCH_SIZE,
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}

t = Trainer(trainer_config)
t.fit(
    model,
    train_dataloader,
    val_dataloader
)

Epoch 1/2


  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

0.6650124192237854
Epoch 2/2


  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

0.6488833427429199


BertForClassification(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): L

<h2> Save and Load model

In [ ]:
t.save("baseline_model.ckpt")

In [ ]:
t = Trainer.load("baseline_model.ckpt")

<h2> Create submission

In [15]:
predictions = t.predict(test_dataloader)

In [16]:
sample_submission = pd.read_csv(os.path.join(PATH, "sample_submission.csv"))
sample_submission["target"] = predictions
sample_submission.head()

,id,target
0,133529667241314002934985813983134580070,2
1,133529693386607632642217219852638190950,1
2,133529737754378640630246272237250379110,1
3,133529756769137644053687294687798459750,1
4,133529828866765532034234504812793265510,1


In [17]:
sample_submission.to_csv("submission.csv", index=False)

In [71]:
n_unfreezed = 0
for name, p in model.named_parameters():
    #print(name)
    if (
        name.startswith("classifier") or
        name.startswith("bert.transformer.layer.5") or
        name.startswith("bert.pooler") or
        name.startswith("bert.encoder.layer.11")
        ):
        
        #p.requires_grad = True
        #n_unfreezed += 1
        1
    else:
        1
        #p.requires_grad = False

print(n_unfreezed)    

0


In [38]:
#config = {
#    "vocab_size" : len(tokenizer.vocab),
#    "data_length": MAX_LEN,
#    "num_classes": 6,
#    "dropout_rate": 0.0,
#    "embedding_size": EMBEDDING_SIZE,
#    'transformer_size': 3,
#    "pretrained_embeddings" : None#  w2v_embedding #None# 
#}
#model = TransformerBasedModel(config)

In [55]:
id2label = {0: 0, 1: 1, 2:2, 3:3,4:4,5:5}
label2id = {0: 0, 1: 1, 2:2, 3:3,4:4,5:5}

In [61]:
from transformers import AutoModelForSequenceClassification, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=6, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

In [10]:
#config = {
#    "vocab_size" : len(tokenizer.vocab),
#    "labels_num" : 6,
#    "max_text_len" : MAX_LEN,
#    "embedding_size" : 128,
#    "n_layers" : 3,
#    "pretrained_embeddings" : None#w2v_embedding
#}    
#model = MyClassificator(config)

<h2>Word2Vec

In [33]:
from dataset import get_data_for_w2v
dataset_for_word_to_vec = get_data_for_w2v(train_data, MAX_LEN, tokenizer)
dataset_for_word_to_vec_2 = get_data_for_w2v(test_data, MAX_LEN, tokenizer)

In [34]:
dataset_for_word_to_vec_full = dataset_for_word_to_vec + dataset_for_word_to_vec_2
print(len(dataset_for_word_to_vec_full))

10734


In [35]:
import gensim
word2vec = gensim.models.Word2Vec(sentences=dataset_for_word_to_vec_full, vector_size=EMBEDDING_SIZE,
                                  window=5, min_count=0, workers=16,
                                  sg=1, epochs=10, null_word=0)

In [36]:
w2v_vectors = np.zeros((len(tokenizer.vocab), EMBEDDING_SIZE)).astype(np.float32)
for new_idx, w2v_idx in enumerate(word2vec.wv.index_to_key):
    w2v_vectors[w2v_idx] = word2vec.wv[new_idx]
w2v_vectors = torch.tensor(w2v_vectors)    
#w2v_vectors

In [37]:
w2v_embedding = torch.nn.Embedding(w2v_vectors.shape[0], w2v_vectors.shape[1], padding_idx=0)
w2v_embedding = w2v_embedding.from_pretrained(w2v_vectors, freeze=True, padding_idx=0)
#w2v_embedding.weight